## Gotta troubleshoot this model and why its not working
To do so, I'm going to visualize what the training and validation sets look like.

In [1]:
import torch
from mimo.models.mimo_unet import MimoUnetModel
from mimo.tasks.sen12tp.sen12tp_datamodule import get_datamodule
from argparse import Namespace
import matplotlib.pyplot as plt
import numpy as np

### First load the Laplace one

In [2]:
args = {
    'seed': 1,
    'checkpoint_path': 'MIMO NDVI Prediction/hhid6a04/checkpoints/last.ckpt',
    'dataset_dir': '/deepskieslab/rnevin/zenodo_data',
    'batch_size': 32,
    'num_loss_function_params': 2,
    'num_subnetworks': 2,
    'filter_base_count': 30,
    'center_dropout_rate': 0.1,
    'final_dropout_rate': 0.1,
    'encoder_dropout_rate': 0.0,
    'core_dropout_rate': 0.0,
    'decoder_dropout_rate': 0.0,
    'loss_buffer_size': 10,
    'loss_buffer_temperature': 0.3,
    'input_repetition_probability': 0.0,
    'batch_repetitions': 1,
    'patch_size': 256,
    'stride': 249,
    'loss': 'laplace_nll',  # Adjust based on your actual loss function
    'weight_decay': 0.0001,
    'learning_rate': 0.0001,
    'num_workers': 30,
    'training_set_percentage': 1.0,
}

args["input"] = ["VV_sigma0", "VH_sigma0"]
args["target"] = ["NDVI"]  # Example target

'''
python scripts/train/train_ndvi.py   --max_epochs 40   --batch_size 32   -t NDVI   -i VV_sigma0   -i VH_sigma0   --project "MIMO_NDVI_Prediction_Gauss"
'''

args = Namespace(**args)  # Convert dictionary to Namespace
dm_laplace = get_datamodule(args)

📦 Total patches loaded: 134080
🗺️ Total unique scenes: 2095
📚 Training scenes: 1865
🧪 Validation scenes: 230
✅ Final train patches: 119360
✅ Final val patches: 14720


Look at the train datamodel.

In [3]:
print(dir(dm_laplace))

['CHECKPOINT_HYPER_PARAMS_KEY', 'CHECKPOINT_HYPER_PARAMS_NAME', 'CHECKPOINT_HYPER_PARAMS_TYPE', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__jit_unused_properties__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_log_hyperparams', '_set_hparams', '_to_hparams_dict', 'allow_zero_length_dataloader_with_multiple_devices', 'batch_size', 'clipping_method', 'dataset_dir', 'drop_last_train', 'end_transform', 'from_datasets', 'hparams', 'hparams_initial', 'load_from_checkpoint', 'load_state_dict', 'model_inputs', 'model_targets', 'name', 'num_workers', 'on_after_batch_transfer', 'on_before_batch_transfer', 'on_exception', 'patch_size', 'pin_memory', 'predict_dataloader', 'prepare_data', 'prepare_data_per_node', 'save_hype

In [4]:
print(dir(dm_laplace.train_dataloader()))

['_DataLoader__initialized', '_DataLoader__multiprocessing_context', '_IterableDataset_len_called', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_auto_collation', '_dataset_kind', '_get_iterator', '_index_sampler', '_is_protocol', '_iterator', 'batch_sampler', 'batch_size', 'check_worker_number_rationality', 'collate_fn', 'dataset', 'drop_last', 'generator', 'multiprocessing_context', 'num_workers', 'persistent_workers', 'pin_memory', 'pin_memory_device', 'prefetch_factor', 'sampler', 'timeout', 'worker_init_fn']


In [5]:
# Now we can use the model to make predictions
train_data_laplace = dm_laplace.train_dataloader()
val_data_laplace = dm_laplace.val_dataloader()

In [6]:
print(dir(train_data_laplace.dataset.ds))

['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', 'clip_transform', 'data_bands', 'end_transform', 'get_image_dataarray', 'load_sample', 'modalities', 'model_inputs', 'model_targets', 'patch_size', 'patches', 'path', 'stride', 'transform']


In [7]:
print("Training patches:", train_data_laplace.dataset.ds.patches[:5])
print("Validation patches:", val_data_laplace.dataset.ds.patches[:5])

Training patches: [({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s2.tif')}, 0, 0), ({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s2.tif')}, 0, 1), ({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s2.tif')}, 0, 2), ({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s2.tif')}, 0, 3), ({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data/train/0_1807/1807_s2.tif')}, 0, 4)]
Validation patches: [({'s1': PosixPath('/deepskieslab/rnevin/zenodo_data/train/1000_662/662_s1.tif'), 's2': PosixPath('/deepskieslab/rnevin/zenodo_data

In [8]:
def extract_patch_identifiers(patches):
    """Extracts unique identifiers for patches."""
    return set(
        (patch[0]['s1'].parent.name, patch[0]['s1'].stem)  # Extract scene and patch ID
        for patch in patches
    )

# Extract unique identifiers for each set
train_patches = extract_patch_identifiers(train_data_laplace.dataset.ds.patches)
val_patches = extract_patch_identifiers(val_data_laplace.dataset.ds.patches)

# Identify overlap
overlapping_patches = train_patches.intersection(val_patches)

# Print results
print(f"Number of overlapping patches: {len(overlapping_patches)}")
print(f"Number of training patches: {len(train_patches)}")
print(f"Number of val patches: {len(val_patches)}")
if overlapping_patches:
    print("Overlapping patches (sample):", list(overlapping_patches)[:5])

Number of overlapping patches: 0
Number of training patches: 1865
Number of val patches: 230
